In [1]:
import pandas

In [2]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");

accuracy = {}; #Grupo de parâmetros: acurácia

In [54]:
#Parâmetros de teste
numReps = 10;
numChanges = 1;
randomSplit = 0.2;


In [8]:
#Geração Aleatória
for n in range(numChanges):
    weights = [random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(), random.randrange(3, 50, 2)]    
    accuracyResults = [];

    for n in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    meanAcc = sum(accuracyResults)/len(accuracyResults)
    print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7])
         )
    accuracy[str(weights)] = meanAcc

Acurácia média em 10 tentativas: 0.6303949894629272

----- PARÂMETROS -----
Número de Vizinhos: 13
Sexo: 0.3965406878829709
Comprimento: 0.16724050132789336
Diâmetro: 0.4787828339016489
Altura: 0.6501962122059938
Peso cheio: 0.811497234028518
Peso vazio: 0.7358234744575249
Peso das vísceras: 0.681448178656213
Peso da concha: 0.6390213700585571


In [ ]:
#Forward selection

def forwardSelection(baseWeights, baseAcc):
    m = 0;
    accs = [];
    for col in abalones.columns[:8]:
        weights = baseWeights.copy();
        weights[m] = 1
        accuracyResults = [];
    
        for num in range(numReps):
            knn = KNeighborsClassifier(n_neighbors = weights[8]);

            #Selecionando índices para teste e treino
            trainingIndexes = [];
            testIndexes = [];
            n = 0;
            for item in abalones["sex"]:
                if random.random() > randomSplit:
                    trainingIndexes.append(n)
                else:
                    testIndexes.append(n)
                n+=1

            #Criando os dataframes de treino e teste e separando os classificadores
            testData = abalones.drop(trainingIndexes);
            testClasses = testData["type"];
            testData = testData.drop(columns=["type"]);

            trainingData = abalones.drop(testIndexes);
            classifications = trainingData["type"]
            trainingData = trainingData.drop(columns=["type"])

            #Aplicando pesos
            n = 0;
            for col in trainingData.columns:
                trainingData[col] = trainingData[col]*weights[n]
                testData[col] = testData[col]*weights[n]
                n+=1

            #Aplicando K-NN
            knn.fit(trainingData, classifications)

            results = knn.predict(testData);

            #Testando acurácia
            n = 0;
            correctPredictions = 0;
            for item in testClasses:
                if item == results[n]:
                    correctPredictions += 1;
                n+=1  

            acc = correctPredictions/len(results)
            accuracyResults.append(acc)

        meanAcc = sum(accuracyResults)/len(accuracyResults)
        '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
        print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
              + "\nSexo: " + str(weights[0]) 
              + "\nComprimento: " + str(weights[1])
              + "\nDiâmetro: " + str(weights[2])
              + "\nAltura: " + str(weights[3])
              + "\nPeso cheio: " + str(weights[4])
              + "\nPeso vazio: " + str(weights[5])
              + "\nPeso das vísceras: " + str(weights[6])
              + "\nPeso da concha: " + str(weights[7]) + "\n\n"
             )'''
        accs.append(meanAcc)
        m+=1
    
    #print("Acurácias finais: " + str(accs) + '\n\n')
    
    #Índice do melhor resultado
    ind = accs.index(max(accs))
    
    #Conferindo se houve melhora nessa recursão, caso não, já retorna a última lista de parâmetros
    if accs[ind] <= baseAcc:
        print("Resultado final: " + str(accs[ind]) + "%")
        return baseWeights
    #Caso haja melhora, a função será chamada novamente, com uma nova base de pesos e com a melhor acurácia
    else:
        newWeights = baseWeights.copy();
        newWeights[ind] = 1;
        return forwardSelection(newWeights, accs[ind])
    
    
for neighs in [5,15,25,50]:
    print(forwardSelection([0,0,0,0,0,0,0,0,neighs], 0))

#bestSet = forwardSelection([0,0,0,0,0,0,0,0,5], 0)

In [12]:
numPop = 50;
population = {}

for n in range(numPop):
    population[n] = [0,0,0,0,0,0,0,0,3]

In [22]:
def generatePop(numPop):
    for m in range(numPop):
        newPop = [];
        for n in range(8):
            newPop.append(random.randrange(101))
        newPop.append(random.randrange(3,100))
        population[m] = newPop;

In [32]:
def checkFitness(weights):
    for num in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    return sum(accuracyResults)/len(accuracyResults)
    '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7]) + "\n\n"
         )'''

In [79]:
def getFitness():
    results = {};
    for n in range(numPop):
        results[n] = checkFitness(population[n]);
        print ("Checando a adequação de "+ str(n+1) + " de " + str(numPop) )
    return results

In [106]:
def selection(fitness):
    #Normalizando a chance de reprodução de acordo com o fitness
    best = max(fitness.values())
    worst =  min(fitness.values())
    for k in fitness.keys():
        fitness[k] = (fitness[k]-worst) / (best-worst)
    
    #Agora, com a tabela normalizada, temos a probabilidade de reprodução entre 0 e 1. Enquanto o tamanho da população não for
    #50% do original, continuamos removendo

    removed = 0;
    tries = 0;
    print(numPop/2)
    while removed < (numPop/2) and tries < 2000:
        ind = random.randrange(numPop);
        print("Índice: " + str(ind) + "\nParâmetros: " + str(population[ind]));
        #Ignorar se já foi removido
        if population[ind] == 0:
            tries+=1
            continue;
        else:
            if random.random() > fitness[ind]:
                print("População removida.")
                removed+=1
                print("Já removidos: " + str(removed))
                population[ind] = 0;
            else:
                tries+=1
    


In [107]:
generatePop(numPop)

In [108]:
population

{0: [65, 58, 73, 89, 65, 0, 77, 97, 90],
 1: [38, 75, 83, 50, 59, 37, 15, 67, 95],
 2: [11, 32, 90, 13, 74, 2, 98, 27, 10],
 3: [63, 15, 61, 88, 51, 61, 54, 42, 39],
 4: [36, 87, 4, 45, 14, 6, 21, 37, 45],
 5: [27, 45, 37, 56, 10, 52, 63, 5, 10],
 6: [20, 92, 39, 47, 88, 92, 55, 64, 11],
 7: [83, 51, 34, 89, 41, 97, 6, 6, 95],
 8: [70, 2, 3, 28, 40, 39, 32, 64, 23],
 9: [36, 56, 76, 100, 48, 31, 57, 18, 70],
 10: [13, 9, 90, 5, 91, 91, 76, 27, 88],
 11: [66, 96, 86, 98, 47, 61, 91, 70, 53],
 12: [21, 21, 12, 71, 34, 23, 8, 18, 10],
 13: [3, 60, 51, 35, 73, 66, 64, 94, 96],
 14: [13, 21, 76, 80, 12, 10, 50, 31, 73],
 15: [5, 25, 86, 2, 71, 2, 40, 9, 54],
 16: [81, 93, 19, 4, 79, 33, 97, 35, 80],
 17: [10, 41, 58, 85, 46, 88, 69, 66, 61],
 18: [33, 16, 11, 8, 72, 92, 32, 27, 49],
 19: [89, 51, 66, 48, 34, 56, 39, 58, 5],
 20: [63, 44, 66, 75, 91, 63, 69, 44, 42],
 21: [52, 62, 32, 9, 13, 12, 49, 89, 74],
 22: [78, 15, 32, 99, 61, 24, 15, 12, 60],
 23: [63, 67, 1, 56, 36, 86, 58, 77, 95],

In [109]:
fitness = getFitness()

Checando a adequação de 1 de 50
Checando a adequação de 2 de 50
Checando a adequação de 3 de 50
Checando a adequação de 4 de 50
Checando a adequação de 5 de 50
Checando a adequação de 6 de 50
Checando a adequação de 7 de 50
Checando a adequação de 8 de 50
Checando a adequação de 9 de 50
Checando a adequação de 10 de 50
Checando a adequação de 11 de 50
Checando a adequação de 12 de 50
Checando a adequação de 13 de 50
Checando a adequação de 14 de 50
Checando a adequação de 15 de 50
Checando a adequação de 16 de 50
Checando a adequação de 17 de 50
Checando a adequação de 18 de 50
Checando a adequação de 19 de 50
Checando a adequação de 20 de 50
Checando a adequação de 21 de 50
Checando a adequação de 22 de 50
Checando a adequação de 23 de 50
Checando a adequação de 24 de 50
Checando a adequação de 25 de 50
Checando a adequação de 26 de 50
Checando a adequação de 27 de 50
Checando a adequação de 28 de 50
Checando a adequação de 29 de 50
Checando a adequação de 30 de 50
Checando a adequaçã

In [110]:
selection(fitness)

25.0
Índice: 27
Parâmetros: [11, 99, 48, 76, 56, 65, 21, 99, 21]
Índice: 3
Parâmetros: [63, 15, 61, 88, 51, 61, 54, 42, 39]
Índice: 21
Parâmetros: [52, 62, 32, 9, 13, 12, 49, 89, 74]
População removida.
Já removidos: 1
Índice: 31
Parâmetros: [0, 21, 25, 82, 42, 42, 52, 87, 49]
Índice: 24
Parâmetros: [31, 13, 94, 93, 79, 18, 56, 74, 21]
Índice: 40
Parâmetros: [19, 55, 10, 29, 43, 56, 93, 99, 58]
População removida.
Já removidos: 2
Índice: 40
Parâmetros: 0
Índice: 10
Parâmetros: [13, 9, 90, 5, 91, 91, 76, 27, 88]
Índice: 42
Parâmetros: [34, 0, 42, 33, 25, 78, 94, 86, 83]
Índice: 30
Parâmetros: [85, 83, 82, 47, 25, 78, 52, 22, 53]
População removida.
Já removidos: 3
Índice: 39
Parâmetros: [80, 63, 23, 20, 51, 15, 92, 63, 41]
População removida.
Já removidos: 4
Índice: 2
Parâmetros: [11, 32, 90, 13, 74, 2, 98, 27, 10]
População removida.
Já removidos: 5
Índice: 3
Parâmetros: [63, 15, 61, 88, 51, 61, 54, 42, 39]
Índice: 16
Parâmetros: [81, 93, 19, 4, 79, 33, 97, 35, 80]
População removida.


In [111]:
population

{0: 0,
 1: [38, 75, 83, 50, 59, 37, 15, 67, 95],
 2: 0,
 3: [63, 15, 61, 88, 51, 61, 54, 42, 39],
 4: [36, 87, 4, 45, 14, 6, 21, 37, 45],
 5: 0,
 6: 0,
 7: [83, 51, 34, 89, 41, 97, 6, 6, 95],
 8: [70, 2, 3, 28, 40, 39, 32, 64, 23],
 9: [36, 56, 76, 100, 48, 31, 57, 18, 70],
 10: [13, 9, 90, 5, 91, 91, 76, 27, 88],
 11: [66, 96, 86, 98, 47, 61, 91, 70, 53],
 12: [21, 21, 12, 71, 34, 23, 8, 18, 10],
 13: [3, 60, 51, 35, 73, 66, 64, 94, 96],
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: [33, 16, 11, 8, 72, 92, 32, 27, 49],
 19: [89, 51, 66, 48, 34, 56, 39, 58, 5],
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: [31, 13, 94, 93, 79, 18, 56, 74, 21],
 25: [67, 99, 6, 88, 75, 66, 87, 58, 93],
 26: [31, 3, 0, 69, 36, 46, 60, 17, 88],
 27: [11, 99, 48, 76, 56, 65, 21, 99, 21],
 28: 0,
 29: 0,
 30: 0,
 31: [0, 21, 25, 82, 42, 42, 52, 87, 49],
 32: 0,
 33: [75, 8, 63, 28, 15, 76, 53, 36, 49],
 34: [82, 88, 51, 32, 84, 7, 39, 27, 6],
 35: 0,
 36: [57, 95, 11, 23, 53, 4, 41, 7, 77],
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 4